In [3]:
import os
import sys
import random
import json
import importlib
import pandas as pd
import nibabel as nib
from datetime import datetime


sys.path.append('/host/verges/tank/data/daniel/00_commonUtils/00_code/genUtils/')
import gen
import bids_naming as names
import dataChecks as check # n.b. now in genUtils, previously in invivoHistology directory

import visUtils
import stitchSurfs as stitch
import projectUtils as prjUtils
import sampleSurfs


#print("Contents:", os.listdir('/host/verges/tank/data/daniel/00_commonUtils/00_code/genUtils/'))



In [ ]:
# parameters

test = False
test_n = 10
verbose = True

demographics_pth = '/host/verges/tank/data/daniel/00_commonUtils/01_demographics/02_combined/demographics_23Jan2026-161349.csv'

dirs_project = {
    'dir_root': '/host/verges/tank/data/daniel/04_inVivoHistology/',
    'dir_data': 'data/',
    'dir_out': 'outputs/'
}

mask_info = {
    'perform': True,
    'dir_root': '/host/verges/tank/data/daniel/04_inVivoHistology/code/resources',
    'maskName': 'Mesial Temporal Mask',
    'maskSuffix': 'mesialTemp',
    'surfs_stitched_mask': "stitch_ctx-glsr_hipp-DK25_05Feb2026_mesialTemporal_mask.gii",
    'surfs_stitched_lbls': "code/resources/stitch_lblVals_ctx-glsr_hipp-DK25_05Feb2026_mesialTemporal_masked.label.gii"
}

if test:
    dirs_project['dir_data'] = os.path.join(dirs_project['dir_data'], 'test/')

features = ["T1map"]

pni_resolution = (0.5, 0.5, 0.5)  # target resolution for PNI 7T scans

lbls_surfs = { # [ctx, hipp]
    'labels': [['pial', 'inner'],
               ['white', 'outer']],
    'surfaces': [['fsLR-32k', 'den-0p5mm']]
}

PNI = {
    'studyName': 'PNI',
    'studyDescrip': '7T',
    'dir_root': '/data/mica3/BIDS_PNI/',
    'dir_deriv': 'derivatives/',
    'dir_fs': 'fastsurfer/',
    'dir_mp': 'micapipe_v0.2.0/',
    'dir_hu': 'hippunfold_v1.3.0/hippunfold/', # update to v2?
}

MICS = {
    'studyName': 'MICs',
    'studyDescrip': '3T',
    'dir_root': '/data/mica3/BIDS_MICs/',
    'dir_deriv': 'derivatives/',
    'dir_fs': 'freesurfer/',
    'dir_mp': 'micapipe_v0.2.0/',
    'dir_hu': 'hippunfold_v1.3.0/hippunfold/', # update to v2?
}




varsOfInterest = ['UID', 'MICS_ID', 'PNI_ID', 'study', 'SES', 'Date', 'sex', 'age', 'grp','grp_detailed']

study_dicts = [PNI, MICS]


In [5]:
# input: demographics datasheet
demo = pd.read_csv(demographics_pth)
print("Shape of demographics:", demo.shape)
print("Columns in demographics:", demo.columns.tolist())
demo.head()

Shape of demographics: (871, 48)
Columns in demographics: ['UID', 'MICS_ID', 'PNI_ID', 'study', 'SES', 'Date', 'gender', 'sex', 'employment', 'language', 'dob', 'education', 'ethnicity', 'lastSeizure', 'handedness', 'Scan_Date (D.M.Y)', 'WeightApprox', 'Handed', 'Employ', 'YoE', 'AssignedSex', 'HeightApprox', 'ASMs  on admission (name, doses (mg per day)', 'Epilepsy diagnosis based on ILAE', 'FDG.PET', 'Duration of admission', 'Surgical resection date and site', 'Invasive explorations (Y/N)', 'Dx at EMU discharge ', 'Baseline MRI (year,results)', 'Epilepsy classification:Focal,Generalized', 'Lateralization of epileptogenic focus', 'Engel classification (seizure outcomes at the 6 month )', 'Epileptogenic focus based on EMU information', 'Drug resistant epilepsy at time of EMU admission', 'Histopatholgy', 'Previous ASMs (name and doses (mg/d)) if applicable prior the current EMU admission', '# of ASM on admission', 'Risk factors for epilepsy', '# of ASMs prior current EMU admission', 'IL

,UID,MICS_ID,PNI_ID,study,SES,Date,gender,sex,employment,language,...,Risk factors for epilepsy,# of ASMs prior current EMU admission,ILAE outcome after surgical resection by 1 yr,Engel classification (seizure outcomes after 1 year from surgical resection),Neuromodulation devices,EMU admission date(dd-mm-yy),# of surgical resection/thermocoagulatin,age,grp,grp_detailed
0,UID0001,NaN,Pilot007,7T,00,11.03.2022,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTRL,CTRL
1,UID0002,NaN,Pilot011,7T,05,28.03.2024,NaN,NaN,Full time student,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTRL,CTRL
2,UID0003,NaN,Pilot012,7T,05,11.04.2024,NaN,NaN,Full time student,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CTRL,CTRL
3,UID0004,HC129,Pilot013,7T,05,18.04.2024,NaN,F,Full time student,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.728953,CTRL,CTRL
4,UID0004,HC129,Pilot013,3T,01,09.07.2024,NaN,F,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.953457,CTRL,CTRL


In [6]:
# QC checks
# for now: extract 7T cases only
do_checks = False
if do_checks:
    importlib.reload(gen)
    importlib.reload(names)
    importlib.reload(check)
    qc_cols = []

    sT_pt = demo[demo['PNI_ID'].isna()==False][varsOfInterest]
    sT_sessions = sT_pt[sT_pt['study'] == '7T']
    print(f'Number of 7T sessions: {len(sT_sessions)}')
    #print(sT_sessions.head())

    print("CHECKING DATA...")
    print(f"\t VOLUMES existence...")
    demo_volCheck, cols_volChk = check.vol_check(PNI, sT_sessions, features, verbose=verbose)
    qc_cols += cols_volChk

    print(f"\t RESOLUTION match {pni_resolution}...")
    demo_res, res_dictList, cols_resChk = check.resolution_check(demo = sT_sessions, study = PNI, res_trgt = pni_resolution, epsilon=0.001, verbose = verbose)
    qc_cols += cols_resChk

    print(f"\t SURFACES present...")
    demo_volSurf_check, cols_procChk = check.proc_check(study = PNI, demo = demo_volCheck, mp_surfaces = mp_surfaces, hu_surfaces = hu_surfaces, verbose=verbose)
    qc_cols += cols_procChk

    # check QC of surfaces and volumes
    # TODO.


    # save
    save_pth = os.path.join(dirs_project['dir_root'], dirs_project['dir_out'], f'demo_QCcols_{gen.fmt_now()}')
    save_item = [demo_volSurf_check, qc_cols]
    demo_volSurf_check.to_parquet(save_pth + ".parquet", index=False)
    # save list
    with open(f"{save_pth}.json", "w") as f:
        json.dump(qc_cols, f, indent=2)
    print(f"Saved DataFrame to: {save_pth}.parquet")
    print(f"Saved QC columns list to: {save_pth}.json")


In [7]:
# filter based on QC columns
reimport = False
if "demo_volSurf_check" not in locals() or reimport:
    common_pth = "/host/verges/tank/data/daniel/04_inVivoHistology/outputs/demo_QCcols_29Jan2026-1206"
    demo_volSurf_check = pd.read_parquet(f"{common_pth}.parquet")
    qc_cols = json.load(open(f"{common_pth}.json"))
    print(f"Loaded DataFrame from: {common_pth}.parquet")
    print(f"Loaded QC columns list from: {common_pth}.json")

# filter out bad QC
qc_cols_present = [col for col in qc_cols if col in demo_volSurf_check.columns]
demo_qc, rm_rows = check.filter_qcCols(demo_volSurf_check, qc_cols)


if test:
    # take random sample of test_n subjects
    demo_qc = demo_qc.sample(n=test_n)

    # take specific row: 
    #demo_qc = demo_qc.loc[demo_qc['PNI_ID'] == 'PNC021']
    
    print(f"TEST MODE: {test_n} rows kept:")
    for idx, row in demo_qc.iterrows():
        print(f"\t{row['UID']}@{row['study']}: {row['MICS_ID']}-{row['PNI_ID']}-{row['SES']}")

demo_qc.head()

Loaded DataFrame from: /host/verges/tank/data/daniel/04_inVivoHistology/outputs/demo_QCcols_29Jan2026-1206.parquet
Loaded QC columns list from: /host/verges/tank/data/daniel/04_inVivoHistology/outputs/demo_QCcols_29Jan2026-1206.json
[filter_qcCols] 19 rows (17 unique participants) removed when filtering QC cols: ['hasVol_T1map', 'properRes', 'mp_proc_pial_fsLR-32k', 'mp_proc_white_fsLR-32k', 'hu_proc_inner_den-0p5mm', 'hu_proc_outer_den-0p5mm']
TEST MODE: 10 rows kept:
	UID0032@7T: HC062-PNC019-04
	UID0070@7T: PX198-PNE018-a1
	UID0015@7T: None-PNC002-01
	UID0063@7T: PX190-PNE011-a1
	UID0094@7T: PX235-PNE033-a1
	UID0031@7T: HC076-PNC018-02
	UID0109@7T: PX081-PNE045-a1
	UID0067@7T: PX200-PNE015-a1
	UID0017@7T: None-PNC004-01
	UID0014@7T: None-PNC001-02


,UID,MICS_ID,PNI_ID,study,SES,Date,sex,age,grp,grp_detailed,hasVol_T1map,properRes,mp_proc_pial_fsLR-32k,mp_proc_white_fsLR-32k,hu_proc_inner_den-0p5mm,hu_proc_outer_den-0p5mm
76,UID0032,HC062,PNC019,7T,04,07.11.2023,M,36.306639,CTRL,CTRL,True,1.0,True,True,True,True
142,UID0070,PX198,PNE018,7T,a1,12.03.2025,M,35.299110,TLE,TLE_L,True,1.0,True,True,True,True
22,UID0015,None,PNC002,7T,01,05.05.2022,None,NaN,CTRL,CTRL,True,1.0,True,True,True,True
131,UID0063,PX190,PNE011,7T,a1,10.01.2025,F,31.545517,EPI_other,EPI_other,True,1.0,True,True,True,True
166,UID0094,PX235,PNE033,7T,a1,29.07.2025,F,41.549624,FLE,FLE_R,True,1.0,True,True,True,True


In [11]:
importlib.reload(prjUtils)
# STITCH
surfs_stitched = prjUtils.stitch_surfs_from_df(dirs_project = dirs_project, study_dicts = study_dicts, 
                              df = demo_qc, lbls_surfs = lbls_surfs, symlink=True)


[stitch_surfs_from_df] Stitching surfaces for 10 rows (unique participant-study-session)...
	UID0032@7T: PNC019-04
	[stitch_surfs_from_df] Stitched surfaces already exist for [ctx] fsLR-32k_pial and [hipp] den-0p5mm_inner -> /host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-PNC019_ses-04/surfs/sub-PNC019_ses-04_hemi-L_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii | /host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-PNC019_ses-04/surfs/sub-PNC019_ses-04_hemi-R_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii. Skipping stitching.
	[stitch_surfs_from_df] Stitched surfaces already exist for [ctx] fsLR-32k_white and [hipp] den-0p5mm_outer -> /host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-PNC019_ses-04/surfs/sub-PNC019_ses-04_hemi-L_ctxSurf-fsLR-32k_ctxLbl-white_hippSurf-den-0p5mm_hippLbl-outer_stitched.surf.gii | /host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI

In [ ]:
# MASK
# as per Paquola, only include neo cortex of the entorhinal, parahippocampal, fusiform
# TODO. USE DKT/VEP-based mask? https://github.com/HuifangWang/VEP_atlas_shared/tree/master
surfs_mask = prjUtils.apply_mask_toStitchedSurfaces(surf_pths = surfs_stitched, 
                                                    mask_pth = os.path.join(mask_info['dir_root'], mask_info['surfs_stitched_mask']), 
                                                    outNameSuffix=mask_info['maskSuffix'])


[apply_mask_toStitchedSurfaces] Masking 40 surfaces...
Saved: /host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-PNC019_ses-04/surfs/sub-PNC019_ses-04_hemi-L_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched_mask-mesialTemp.surf.gii
Vertices: 37741 → 8477
Saved: /host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-PNC019_ses-04/surfs/sub-PNC019_ses-04_hemi-R_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched_mask-mesialTemp.surf.gii
Vertices: 37741 → 8477
Saved: /host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-PNC019_ses-04/surfs/sub-PNC019_ses-04_hemi-L_ctxSurf-fsLR-32k_ctxLbl-white_hippSurf-den-0p5mm_hippLbl-outer_stitched_mask-mesialTemp.surf.gii
Vertices: 37741 → 8477
Saved: /host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-PNC019_ses-04/surfs/sub-PNC019_ses-04_hemi-R_ctxSurf-fsLR-32k_ctxLbl-white_hippSurf-den-0p5mm_hippLbl-outer_stitched_mask-mesialTemp.surf.gii
Vertices: 3774

In [24]:
importlib.reload(prjUtils)
# EQUIVOL SURFS
prjUtils.sample_stitchedSurfs_from_df(df=demo_qc, study_dicts=study_dicts, dirs_project=dirs_project, nSurfs=16, ctx_surf="fsLR-32k", hipp_surf="den-0p5mm", mask_info = mask_info, verbose=True)


[sample_stitchedSurfs_from_df] Sampling 16 equi-volume surfaces from stitched surfaces for 10 rows (unique participant-study-session)...
	UID0032@7T: PNC019-04
	Surfaces L: ['/host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-PNC019_ses-04/surfs/sub-PNC019_ses-04_hemi-L_equivol-1of16.surf.gii', '/host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-PNC019_ses-04/surfs/sub-PNC019_ses-04_hemi-L_equivol-2of16.surf.gii', '/host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-PNC019_ses-04/surfs/sub-PNC019_ses-04_hemi-L_equivol-3of16.surf.gii', '/host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-PNC019_ses-04/surfs/sub-PNC019_ses-04_hemi-L_equivol-4of16.surf.gii', '/host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-PNC019_ses-04/surfs/sub-PNC019_ses-04_hemi-L_equivol-5of16.surf.gii', '/host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-PNC019_ses-04/surfs/sub-PNC019_ses-04_hemi-L_equivol-6of16.surf.gii'

In [ ]:
# SAMPLE VOLUME
# SMOOTH

[sample_stitchedSurfs_from_df] Sampling 16 equi-volume surfaces from stitched surfaces for 1 rows (unique participant-study-session)...
	UID0034@7T: PNC021-01
	Surfaces L: ['/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/surfs/sub-PNC021_ses-01_hemi-L_equivol-1of16.surf.gii', '/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/surfs/sub-PNC021_ses-01_hemi-L_equivol-2of16.surf.gii', '/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/surfs/sub-PNC021_ses-01_hemi-L_equivol-3of16.surf.gii', '/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/surfs/sub-PNC021_ses-01_hemi-L_equivol-4of16.surf.gii', '/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/surfs/sub-PNC021_ses-01_hemi-L_equivol-5of16.surf.gii', '/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/surfs/sub-PNC021_ses-01_hemi-L_equivol-6of16.surf.gii', '/host/verges/tank/data/danie

In [27]:
importlib.reload(visUtils)
importlib.reload(prjUtils)
importlib.reload(check)
# get bash command to visualize surfaces and volumes
visualization_params = {
    'add_stitchedSurf': True,
    'add_sampledSurfs': True,
    'nSurfs': 16,
    'ctx_surf': "fsLR-32k",
    'hipp_surf': "den-0p5mm",
    'equiVol_str': "equivol",
    'mask':{
        'applied': True,
        'suffix': mask_info['maskSuffix'],
    },
    'colour': {
        'do': False,
        'outer': 'red',
        'inner': 'blue'
    }
}

for pt in demo_qc.itertuples():
    uid = pt.UID
    study = pt.study
    ses = pt.SES
    mics_id = pt.MICS_ID
    pni_id = pt.PNI_ID

    if study == '7T':
        id = pni_id
        study_dict = next(sd for sd in study_dicts if sd['studyName'] == 'PNI')
    elif study == '3T':
        id = mics_id
        study_dict = next(sd for sd in study_dicts if sd['studyName'] == 'MICs')
    else:
        print(f"[main] WARNING: study {study} not recognized. Skipping subject {uid}...")
        continue

    mp_root = study_dict['dir_root'] + study_dict['dir_deriv'] + study_dict['dir_mp']
    hu_root = study_dict['dir_root'] + study_dict['dir_deriv'] + study_dict['dir_hu']

    # get paths
    vol_pth = names.get_volPath(study = study_dict, id = id, ses = ses, volName = 'T1map')[0]
    
    if visualization_params['add_stitchedSurf'] or visualization_params['add_sampledSurfs']:
        dir_subjectData = os.path.join(prjUtils.get_path_data(dirs_project, studyName = study_dict['studyName'], id = id, ses = ses), 'surfs')
        prjUtils.make_dir(dir_subjectData)
        print(f"Subject data directory: {dir_subjectData}")
    
    surface_combinations = prjUtils.iterate_labels(lbls_surfs)
    print(surface_combinations)
    
    for surfs, lbls in surface_combinations:
        surf_pths = []
        ctx_surf = surfs[0]
        hipp_surf = surfs[1]

        ctx_lbl = lbls[0]
        hipp_lbl = lbls[1]

        print(f"Combination: {ctx_surf}-{ctx_lbl} | {hipp_surf}-{hipp_lbl}")

        mp_surfs = names.get_surf_pth(root = mp_root, sub = id, ses = ses, lbl=ctx_lbl, surf=ctx_surf, verbose = False)
        hu_surfs = names.get_surf_pth(root = hu_root, sub = id, ses = ses, lbl=hipp_lbl, surf=hipp_surf, verbose = False)
        print(f"MP surfaces: {mp_surfs}")
        print(f"HU surfaces: {hu_surfs}")
        surf_pths += mp_surfs
        surf_pths += hu_surfs

        if visualization_params['add_stitchedSurf']:
            print("\tGetting stitched surf names...")
            stched_name_l, stched_name_r = prjUtils.get_names_stitchSurf(id=id, ses=ses, ctx_lbl=ctx_lbl, ctx_surf=ctx_surf, hipp_lbl=hipp_lbl, hipp_surf=hipp_surf)
            
            if visualization_params['mask']['applied']:
                print(f"\t\tTaking stitched surfaces with mask applied: {visualization_params['mask']['suffix']}")
                stched_name_l = stched_name_l.replace(".surf.gii", f"_mask-{visualization_params['mask']['suffix']}.gii")
                stched_name_r = stched_name_r.replace(".surf.gii", f"_mask-{visualization_params['mask']['suffix']}.gii")
            print(f"\t\tStitched names: {stched_name_l}, {stched_name_r}")
            surf_pths += [os.path.join(dir_subjectData, stched_name_l), 
                        os.path.join(dir_subjectData, stched_name_r)]

        if visualization_params['add_sampledSurfs']:
            print("\tGetting sampled surf names...")
            outNamePrefix = f"{gen.fmt_id_ses(id,ses)}"
            baseNames = [f"{outNamePrefix}_hemi-L_{visualization_params['equiVol_str']}",
                         f"{outNamePrefix}_hemi-R_{visualization_params['equiVol_str']}"]
            
            # search for all files containing either base name in the dir_subjectData
            for bn in baseNames:
                print(f"\t\tLooking for basename: {bn}")
                matched_files = [f for f in os.listdir(dir_subjectData) if bn in f and f"of{visualization_params['nSurfs']}" in f and f.endswith('.surf.gii')]
                print(f"\t\tMatched files {len(matched_files)}: {matched_files}")
                surf_pths += [os.path.join(dir_subjectData, f) for f in matched_files if f"of{visualization_params['nSurfs']}" in f]

    # generate visualization command
    # check that all files exist
    surf_exist = check.check_paths_exist(surf_pths)
    surf_pths = [surf_pth for surf_pth, exist in zip(surf_pths, surf_exist) if exist]
    print(visUtils.getCMD_freeView(surf_pths=surf_pths, vol_pth=vol_pth, 
                                colour = visualization_params['colour']))

        

Subject data directory: /host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-PNC019_ses-04/surfs
[(['fsLR-32k', 'den-0p5mm'], ['pial', 'inner']), (['fsLR-32k', 'den-0p5mm'], ['white', 'outer'])]
Combination: fsLR-32k-pial | den-0p5mm-inner
MP surfaces: ('/data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC019/ses-04/surf/sub-PNC019_ses-04_hemi-L_space-nativepro_surf-fsLR-32k_label-pial.surf.gii', '/data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC019/ses-04/surf/sub-PNC019_ses-04_hemi-R_space-nativepro_surf-fsLR-32k_label-pial.surf.gii')
HU surfaces: ('/data/mica3/BIDS_PNI/derivatives/hippunfold_v1.3.0/hippunfold/sub-PNC019/ses-04/surf/sub-PNC019_ses-04_hemi-L_space-T1w_den-0p5mm_label-hipp_inner.surf.gii', '/data/mica3/BIDS_PNI/derivatives/hippunfold_v1.3.0/hippunfold/sub-PNC019/ses-04/surf/sub-PNC019_ses-04_hemi-R_space-T1w_den-0p5mm_label-hipp_inner.surf.gii')
	Getting stitched surf names...
		Taking stitched surfaces with mask applied: mesialTemp
		Stitched 

In [ ]:


# following steps should be able to be performed on data from any source (PNI, MICs, AHEAD, BigBrain, etc.)

# save surfaces generated at different depths to study directory
# stitch cortical and hippocampal surfaces together. NOTE. Jordan code
# remove vertices not in mesial temporal lobe (choose cut-off limit as per Paquola 2020)
# assign each vertex values along anterior-posterior, and allo/neo-cortical axes
# project features onto surfaces


In [ ]:
# analyses

In [20]:
dir_subjectData = "/host/verges/tank/data/daniel/04_inVivoHistology/data/test/PNI/sub-PNA004_ses-a1"
bn = "sub-PNA004_ses-a1_hemi-R_equivol"
matched_files = [f for f in os.listdir(dir_subjectData) if bn in f]
print(f"Matched files: {matched_files}")

Matched files: []


In [21]:
os.listdir(dir_subjectData)

['surfs']